In [27]:
import os
import pandas as pd
from geo import get_place_list, get_place_data
from params import DATA_DIR

Load some data

* `all_geos` is a list of all geographies mentioned in the the geography tree csv file
* `population_data` is population estimates for a (some of) the geographies
* `council_tax_data` is estimates of people in receipt of council tax support
* `clif_data` is data bout children living in poverty 

In [28]:
population_data = pd.read_csv(f'{DATA_DIR}/population-estimates/population-estimates.csv')

In [29]:
gva = pd.read_csv(f'{DATA_DIR}/gva/gva.csv')

In [30]:
area_of_place =pd.read_csv(f'{DATA_DIR}/geo/area_of_places.csv')

In [31]:
households = pd.read_csv(f'{DATA_DIR}/households/households.csv')

In [32]:
council_tax_data = pd.read_csv(f'{DATA_DIR}/council-tax-support/council-tax-support.csv')
council_tax_data = council_tax_data[council_tax_data.date == max(council_tax_data.date)]

In [33]:
clif_data = pd.read_csv(f'{DATA_DIR}/clif/clif_REL.csv')
clif_data = clif_data[
    (clif_data['Age of Child (years and bands)'] == 'Total') &
    (clif_data['Gender of Child'] == 'Total') &
    (clif_data['Family Type'] == 'Total') &
    (clif_data['Work Status'] == 'Total') &
    (clif_data.Year == max(clif_data.Year))
]
clif_data

,Year,Age of Child (years and bands),Gender of Child,Family Type,Work Status,variable_name,geography_code,value
134,2021/22,Total,Total,Total,Total,children_in_low_income,E06000047,27178.0
296,2021/22,Total,Total,Total,Total,children_in_low_income,E06000005,6070.0
458,2021/22,Total,Total,Total,Total,children_in_low_income,E06000001,5870.0
620,2021/22,Total,Total,Total,Total,children_in_low_income,E06000002,13432.0
782,2021/22,Total,Total,Total,Total,children_in_low_income,E06000057,13562.0
...,...,...,...,...,...,...,...,...
60236,2021/22,Total,Total,Total,Total,children_in_low_income,Mid and East Antrim,5687.0
60398,2021/22,Total,Total,Total,Total,children_in_low_income,Mid Ulster,8079.0
60560,2021/22,Total,Total,Total,Total,children_in_low_income,"Newry, Mourne and Down",10990.0
60722,2021/22,Total,Total,Total,Total,children_in_low_income,Unknown,0.0


In [34]:
number_of_persons = pd.read_csv(f'{DATA_DIR}/population-estimates/population-estimates-ages.csv')
number_of_children = number_of_persons[number_of_persons.age_band.isin(['0-4', '5-10', '11-15', '16-19'])]
number_of_children = number_of_children.groupby('geography_code').sum(numeric_only=True).reset_index()
number_of_children['variable_name'] = 'number_of_children'
number_of_children

,geography_code,date,value,variable_name
0,E05000650,8080,3390,number_of_children
1,E05000651,8080,2530,number_of_children
2,E05000652,8080,3722,number_of_children
3,E05000653,8080,2813,number_of_children
4,E05000654,8080,4875,number_of_children
...,...,...,...,...
1315,E47000003,8084,586078,number_of_children
1316,E47000004,8084,346827,number_of_children
1317,E47000006,8084,159599,number_of_children
1318,E47000010,8084,247569,number_of_children


In [35]:
savings = pd.read_csv(f'{DATA_DIR}/hbai/Savings and Investments of Adults in the Family of the Individual.csv')
savings = savings[savings['Financial Year'] == max(savings['Financial Year'])]
savings = savings[savings.variable_name == 'In low income (below threshold)']
savings = savings[savings['Savings and Investments of Adults in the Family of the Individual'] == 'No savings']
savings.variable_name = 'households_low_income_no_savings'
savings
#save each geog_code into 'src/place/EXXXXXX/_data/' and then can build the visualisations in place.njk on a conidtional statement if that data exists.

,Financial Year,geography_name,geography_code,Savings and Investments of Adults in the Family of the Individual,variable_name,value
1321,2021-22,North East,E12000001,No savings,households_low_income_no_savings,103723.0
1329,2021-22,North West,E12000002,No savings,households_low_income_no_savings,403448.0
1337,2021-22,Yorkshire and The Humber,E12000003,No savings,households_low_income_no_savings,255022.0


Concatenate the loaded data, then pivot into a table with a line per geography code. Filter this to only include geographies that are in the canonical list of areas.

In [36]:
place_data = pd.concat([
    population_data,
    area_of_place,
    gva,
    households,
    council_tax_data,
    clif_data,
    number_of_children,
    savings
]).pivot(index='geography_code', columns='variable_name', values='value')
place_data = place_data.loc[place_data.index.isin(get_place_list())]
place_data = place_data.merge(get_place_data(), left_index=True, right_index=True, how='outer')
place_data.index.name = 'geography_code'
place_data

,Area in sq km,GVA,Number of households,Number of persons,children_in_low_income,households_low_income_no_savings,number_of_children,pensioners,working_age,ancestors,parents,children,name,type
geography_code,,,,,,,,,,,,,,
E05000650,NaN,NaN,NaN,14133.0,NaN,NaN,3390.0,NaN,NaN,"[E08000001, E47000001, E12000002, E12999901]",[E08000001],[],Astley Bridge,WD22
E05000651,NaN,NaN,NaN,11331.0,NaN,NaN,2530.0,NaN,NaN,"[E08000001, E47000001, E12000002, E12999901]",[E08000001],[],Bradshaw,WD22
E05000652,NaN,NaN,NaN,14078.0,NaN,NaN,3722.0,NaN,NaN,"[E08000001, E47000001, E12000002, E12999901]",[E08000001],[],Breightmet,WD22
E05000653,NaN,NaN,NaN,13503.0,NaN,NaN,2813.0,NaN,NaN,"[E08000001, E47000001, E12000002, E12999901]",[E08000001],[],Bromley Cross,WD22
E05000654,NaN,NaN,NaN,16828.0,NaN,NaN,4875.0,NaN,NaN,"[E08000001, E47000001, E12000002, E12999901]",[E08000001],[],Crompton,WD22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E47000003,2022.032423,57909.0,1035190.0,2349987.0,NaN,NaN,586078.0,NaN,NaN,"[E12000003, E12999901]",[E12000003],"[E08000032, E08000033, E08000034, E08000035, E...",West Yorkshire,CAUTH22
E47000004,722.669297,33598.0,729960.0,1551722.0,NaN,NaN,346827.0,NaN,NaN,"[E12000002, E12999901]","[E12000002, E12000002]","[E08000011, E08000012, E08000013, E08000014, E...",Liverpool City Region,CAUTH22
E47000006,801.149717,13951.0,317080.0,678173.0,NaN,NaN,159599.0,NaN,NaN,"[E12000001, E12999901]",[E12000001],"[E06000001, E06000002, E06000003, E06000004, E...",Tees Valley,CAUTH22


Finally, write the data to an interim parquet and json file for later usage.

In [37]:
os.makedirs(f'{DATA_DIR}/interim/', exist_ok=True)
place_data.to_parquet(f'{DATA_DIR}/interim/place_data.parquet')
place_data.reset_index().to_json(f"{DATA_DIR}/interim/place_data.json", orient="records")